# GGA 梯度

这份笔记我们讨论 GGA 梯度．GGA 与 HF 的过程非常相似，仅仅多出交换相关能；因此，类似于 $h_{\mu \nu}^{A_t}$ 等项是完全一致的．但也正因为多出的交换相关能，计算过程就变得相对复杂．

这一份笔记中，我们使用 B3LYP 密度；并且不涉及非自洽计算．同时，我们使用到新的 GGAHelper 类与 GridHelper, GGAKernelHelper 类．GGA 帮手类 GGAHelper 继承于 HFHelper，因此变量的含义一致，只是实现通过 GGA 的格点积分．

GridHelper 类则计算了与泛函无关的格点，包括格点信息、原子轨道格点与密度格点．GGAKernelHelper 类则计算与泛函有关的格点，主要是泛函核在特定密度下的结果．

In [1]:
import numpy as np
from pyscf import scf, gto, lib, grad, hessian, dft
import pyscf.hessian.rks
import pyscf.grad.rks
from functools import partial

from utilities import val_from_fchk, NumericDiff
from hessian import HFHelper, GGAHelper

pyximporteinsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.einsum_path = partial(np.einsum_path, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.set_printoptions(5, linewidth=120, suppress=True)

In [2]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

def mol_to_grids(mol):
    grids = dft.gen_grid.Grids(mol)
    grids.atom_grid = (75, 302)
    grids.becke_scheme = dft.gen_grid.stratmann
    grids.prune = None
    grids.build()
    return grids

grids = mol_to_grids(mol)

nmo = nao = mol.nao
natm = mol.natm
nocc = mol.nelec[0]
nvir = nmo - nocc
so = slice(0, nocc)
sv = slice(nocc, nmo)
sa = slice(0, nmo)

GGAHelper 相对于 HFHelper 类，多出 `grdh` GridHelper 类的信息，`kerh` KernelHelper 类的信息，以及 `cx` 交换积分系数．其余的项仍然与 HF 情形一致．

In [3]:
ggah = GGAHelper(mol, "b3lypg", grids, init_scf=True)
grdh     = ggah.get_grdh()
kerh     = ggah.get_kerh()
cx       = ggah.cx

/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/utilities/grid_helper.py:19: UserWarning: GridHelper is considered memory consuming!
  warnings.warn("GridHelper is considered memory consuming!")


In [4]:
%%capture
ggah.get_grad()
ggah.get_hess()
scf_eng  = ggah.scf_eng
scf_grad = ggah.scf_grad
scf_hess = ggah.scf_hess
C        = ggah.C
Co       = ggah.Co
Cv       = ggah.Cv
e        = ggah.e
eo       = ggah.eo
ev       = ggah.ev
D        = ggah.D
F_0_ao   = ggah.F_0_ao
F_0_mo   = ggah.F_0_mo
H_0_ao   = ggah.H_0_ao
H_0_mo   = ggah.H_0_mo
eri0_ao  = ggah.eri0_ao
eri0_mo  = ggah.eri0_mo
mo_occ   = ggah.mo_occ
H_1_ao   = ggah.H_1_ao
H_1_mo   = ggah.H_1_mo
S_1_ao   = ggah.S_1_ao
S_1_mo   = ggah.S_1_mo
F_1_ao   = ggah.F_1_ao
F_1_mo   = ggah.F_1_mo
eri1_ao  = ggah.eri1_ao
H_2_ao   = ggah.H_2_ao
H_2_mo   = ggah.H_2_mo
S_2_ao   = ggah.S_2_ao
S_2_mo   = ggah.S_2_mo
eri2_ao  = ggah.eri2_ao
B_1      = ggah.B_1
U_1      = ggah.U_1
Xi_2     = ggah.Xi_2
Ax0_Core = ggah.Ax0_Core
Ax1_Core = ggah.Ax1_Core
mol_slice = ggah.mol_slice

## PySCF 一阶、二阶梯度计算

我们在这里先不使用 GGAHelper 提供的 `scf_grad` 与 `scf_hess` 来计算结果；因此使用 `scf_grad_` 与 `scf_hess_` 来区分这两者．PySCF 的 GGA 梯度性质计算结果与 Gaussian 尽管相近，但并不能很精确地核准到单浮点数精度．

一阶梯度计算代码如下：

In [5]:
scf_grad_ = grad.rks.Gradients(scf_eng)
scf_grad_.kernel()

array([[-0.11105,  0.01396,  0.00386],
       [ 0.01289,  0.74497,  0.01315],
       [ 0.0924 ,  0.00269,  0.01866],
       [ 0.00576, -0.76162, -0.03567]])

In [6]:
np.allclose(
    scf_grad_.de.ravel(),
    val_from_fchk("Cartesian Gradient", "include/gga_grad/gga_grad.fchk"),
    atol=1e-5
)

True

由于 GGA 相对于 SCF 而言还多出了格点积分的计算，因此耗时会变得非常大．为计算数值梯度，要花去很多时间．在这里我们仅仅列出代码．

In [7]:
# %%time
# eng_diff = NumericDiff(mol, lambda mol : GGAHelper(mol, "b3lypg", grids).eng).get_numdif()

In [8]:
# np.allclose(eng_diff, scf_grad.de)

二阶梯度计算代码如下：

In [9]:
scf_hess_ = hessian.rks.Hessian(scf_eng)
scf_hess_.kernel();

In [10]:
d_hess = mol.natm * 3
np.allclose(
    scf_hess.de.swapaxes(1, 2).reshape(d_hess, d_hess)[np.tril_indices(d_hess)],
    val_from_fchk("Cartesian Force Constants", "include/gga_grad/gga_grad.fchk"),
    atol=1e-4, rtol=1e-3
)

True

## 格点与泛函的核坐标导数定义

我们单列一小节对核坐标导数的相关符号与公式推导作定义．此前我们已经对不少 [与格点相关的定义](basic_gga.ipynb#格点与泛函相关定义) 作介绍，包括普通格点、轨道与密度格点、泛函核格点，以及轨道与密度格点在电子坐标下的梯度．但是，轨道与密度格点在核坐标下的梯度我们还没有作过定义．

<div class="alert alert-info">

**记号说明**

* $\phi_{t \mu_A}$ 表示 $\phi_{\mu}$ 在电子坐标分量 $t$ 下的导数；但若 $\mu$ 不是原子 $A$ 的原子轨道，那么该值为零，或称无意义．注意到这里的 $t$ 表示电子坐标分量，但通常是从原子坐标分量 $A_t$ 衍生而来．

* $\phi^{A_t}_{\mu} = \partial_{A_t} \phi_{\mu}$

* $\phi^{A_t}_{r \mu} = \partial_{A_t} \phi_{r \mu}$，其它高阶梯度类同

* $\rho^{A_t} = \partial_{A_t} \rho$，其它梯度记号类同于原子轨道的记号

* 以后为了方便起见，有时会使用 $X_{\kappa \lambda}^{A_t} = \frac{1}{2} \partial_{A_t}^\mathrm{U} D_{\kappa \lambda} = U_{mk} (C_{\kappa m} C_{\lambda k} + C_{\kappa k} C_{\lambda m})$，容易知道 $X_{\kappa \lambda}^{A_t}$ 是对称矩阵；这里 $m$ 代表全轨道．$X_{\kappa \lambda}$ 记号也可能代表任意矩阵．

</div>

下面我们简单说明核坐标导数与电子坐标导数之间的关系．我们会用到前两份笔记中提及的技巧．

我们先把记号补充成完全的状态并作一定程度的简化．那么

$$
\phi_\mu^{A_t} = \partial_{A_t} \phi_\mu = \partial_{A_t} \phi_{\mu} (\boldsymbol{r} - \boldsymbol{A}) = \partial_{A_t} \phi_{\mu} (t - A_t) = - \partial_{t} \phi_{\mu_A} (t - A_t) = - \partial_{t} \phi_{\mu_A} = - \phi_{t \mu_A}
$$

因此，很多时候我们都能把 $\partial_{A_t}$ 的记号当作 $- \partial_t$，并且与 $t$ 求导有关的原子轨道的角标只能在原子 $A$ 上．这种求导方式对于高阶梯度而言也是类似的．

现在我们考虑对密度求核坐标梯度．

$$
\rho^{A_t} = \partial_{A_t} \rho = \partial_{A_t} (D_{\mu \nu} \phi_{\mu} \phi_{\nu}) = - D_{\mu \nu} (\phi_{t \mu_A} \phi_{\nu} + \phi_{\mu} \phi_{t \nu_A}) = - 2 D_{\mu \nu} \phi_{t \mu_A} \phi_{\nu}
$$

求取 $\rho_{r}^{A_t}$ 的过程是相同的．这些变量可以通过 `gdh.A_rho_1` $\rho^{A_t}$, `gdh.A_rho_2` $\rho^{A_t}_r$ 得到．

我们看到，在上面推导中利用到了 $\partial_{A_t} D_{\mu \nu} = 0$．事实上，对一个密度的关于 $A_t$ 的完整偏导还需要

$$
\partial_{A_t}^\mathrm{U} \rho = \partial_{A_t}^\mathrm{U} D_{\mu \nu} \cdot \phi_\mu \phi_\nu = 4 U_{pi}^{A_t} C_{\mu p} C_{\nu i} \phi_\mu \phi_\nu = 2 X_{\mu \nu}^{A_t} \phi_\mu \phi_\nu
$$

这与上一份笔记中经常看到的推导过程是一样的．

类似地，

$$
\partial_{A_t}^\mathrm{U} \rho_r = \partial_{A_t}^\mathrm{U} D_{\mu \nu} \cdot (\phi_{r \mu} \phi_\nu + \phi_\mu \phi_{r \nu}) = 4 U_{pi}^{A_t} C_{\mu p} C_{\nu i} (\phi_{r \mu} + \phi_\nu + \phi_\mu + \phi_{r \nu}) = 8 X_{\mu \nu}^{A_t} \phi_{r \mu} \phi_\nu
$$

上式用到了 $X_{\mu \nu}^{A_t}$ 的对称性质．

我们最后简单讨论一下 $f_\rho$ 的导数的推导过程；这个过程对于理解 $f_\gamma, f_{\rho \gamma}$ 等是完全一致的．回顾到

$$
\gamma = \rho_t \rho_t
$$

它可能从一开始来看不太直观，因为我们可能更习惯 $\gamma = \nabla \rho \cdot \nabla \rho$，但使用上面的公式在我看来会对理解公式推导更为清晰．

我们首先来看完整导数：

\begin{align}
\frac{\partial f_\rho}{\partial A_t} &= \frac{\partial f_\rho}{\partial \rho} \frac{\partial \rho}{\partial A_t} + \frac{\partial f_\rho}{\partial \gamma} \frac{\partial \gamma}{\partial \rho_t} \frac{\partial \rho_t}{\partial A_t} \\
&= f_{\rho \rho} \cdot (\partial_{A_t} + \partial_{A_t}^\mathrm{U}) \rho + f_{\rho \gamma} \cdot 2 \rho_r \cdot (\partial_{A_t} + \partial_{A_t}^\mathrm{U}) \rho_r
\end{align}

我们遂将 $\partial_{A_t}$ 与 $\partial_{A_t}^\mathrm{U}$ 的部分区分开来，并定义

$$
\partial_{A_t} f_{\rho} = f_{\rho \rho} \rho^{A_t} + 2 f_{\rho \gamma} \rho_r \rho^{A_t}_r
$$

以及

$$
\partial_{A_t}^\mathrm{U} f_\rho = f_{\rho \rho} \cdot \partial_{A_t}^\mathrm{U} \rho + 2 f_{\rho \gamma} \rho_r \cdot \partial_{A_t}^\mathrm{U} \rho_r
$$

我们无法对上式直接程序化；因此我们还需要进一步作展开．我们首先要回顾到

$$
\rho_r = \partial_r \rho = 2 D_{\mu \nu} \phi_{r \mu} \phi_\nu
$$

那么，

\begin{align}
\partial_{A_t}^\mathrm{U} f_\rho &= 4 f_{\rho \rho} U_{pi}^{A_t} C_{\mu p} C_{\nu i} \phi_\mu \phi_\nu + 8 f_{\rho \gamma} \rho_r \cdot U_{pi}^{A_t} ( C_{\mu p} C_{\nu i} + C_{\mu i} C_{\nu p} ) \phi_{r \mu} \phi_\nu \\
&= 2 f_{\rho \rho} X_{\mu \nu}^{A_t} \phi_\mu \phi_\nu + 8 f_{\rho \gamma} \rho_r X_{\mu \nu}^{A_t} \phi_{r \mu} \phi_\nu
\end{align}

<div class="alert alert-info">

**任务**

* 请解释为何下述代码执行结果是 False；或者说，为何上式 RHS 的第二项不能化简为 $16 f_{\rho \gamma} \rho_r U_{pi}^{A_t} C_{\mu p} C_{\nu i} \phi_{r \mu} \phi_\nu$？

</div>

In [11]:
X = np.random.random((nmo, nocc))
np.allclose(
    + 8 * np.einsum("g, rg, pi, up, vi, rgu, gv -> g", kerh.frg, grdh.rho_1, X, C, Co, grdh.ao_1, grdh.ao_0)
    + 8 * np.einsum("g, rg, pi, ui, vp, rgu, gv -> g", kerh.frg, grdh.rho_1, X, Co, C, grdh.ao_1, grdh.ao_0),
    16 * np.einsum("g, rg, pi, up, vi, rgu, gv -> g", kerh.frg, grdh.rho_1, X, C, Co, grdh.ao_1, grdh.ao_0)
)

False

$\partial_{A_t} f_{\rho}$ 与 $\partial_{A_t}^\mathrm{U} f_\rho$ 的程序化将会在具体例子中再进行描述；我们不在这里生成它们．

## 实现参考：一阶导数性质

### GGA 一阶梯度 $\frac{\partial}{\partial A_t} E_\mathrm{elec}$

一阶能量梯度在 GGA 的各种一阶梯度中应认为是最容易实现的一个．如果不考虑公式推导，我们假定

$$
\frac{\partial}{\partial A_t} E_\mathrm{elec} = h_{\mu \nu}^{A_t} D_{\mu \nu} + \frac{1}{2} (\mu \nu | \kappa \lambda)^{A_t} D_{\mu \nu} D_{\kappa \lambda} - \frac{c_\mathrm{x}}{4} (\mu \kappa | \nu \lambda)^{A_t} D_{\mu \nu} D_{\kappa \lambda} + \partial_{A_t} (f \rho) - 2 S_{\mu \nu} C_{\mu i} \varepsilon_i C_{\nu i}
$$

事实上，这几乎只比 RHF 的一阶梯度多了交换相关能的梯度．同时，我们也应当能够推导 (注意到 $f_\rho = \partial_{\rho} (f \rho)$ 与 $f_\gamma = \partial_{\gamma} (f \rho)$)

$$
\partial_{A_t} (f \rho) = f_\rho \rho^{A_t} + 2 f_\gamma \rho_r \rho_r^{A_t}
$$

因此，电子能量的一阶梯度的程序就很容易地给出：

In [12]:
np.allclose(
    + np.einsum("Atuv, uv -> At", H_1_ao, D)
    + 0.5 * np.einsum("Atuvkl, uv, kl -> At", eri1_ao, D, D)
    - 0.25 * cx * np.einsum("Atukvl, uv, kl -> At", eri1_ao, D, D)
    + np.einsum("g, Atg -> At", kerh.fr, grdh.A_rho_1)  # GGA Contrib
    + 2 * np.einsum("g, rg, Atrg -> At", kerh.fg, grdh.rho_1, grdh.A_rho_2)  # GGA Contrib
    - 2 * np.einsum("Atuv, ui, i, vi -> At", S_1_ao, Co, eo, Co),
    scf_grad.grad_elec()
)

True

### 交换泛函势一阶 Skeleton 导数 $v_{\mu \nu}^{\mathrm{xc}, A_t} [\rho]$

我们简单回顾一下 [交换相关势](basic_gga.ipynb#实现参考) 的表达方式：

\begin{align}
v_{\mu \nu}^\mathrm{xc} [\rho] &= f_\rho \phi_\mu \phi_\nu + 2 f_\gamma \rho_r (\phi_{r \mu} \phi_{\nu} + \phi_{\mu} \phi_{r \nu}) \\
&= \frac{1}{2} f_\rho \phi_\mu \phi_\nu + 2 f_\gamma \rho_r \phi_{r \mu} \phi_{\nu} + \mathrm{interchange} (\mu, \nu)
\end{align}

现在，我们需要对上式进行 Skeleton 导数 $\partial_{A_t}$ 的求取：

\begin{align}
%%
v_{\mu \nu}^{\mathrm{xc}, A_t} [\rho]
%
&=
\frac{1}{2} \partial_{A_t} f_\rho \cdot \phi_\mu \phi_\nu
+ 2 \partial_{A_t} f_\gamma \cdot \rho_r \phi_{r \mu} \phi_{\nu}
+ 2 f_\gamma \cdot \partial_{A_t} \rho_r \cdot \phi_{r \mu} \phi_{\nu}
%
\\ &\quad \mathrel+
f_\rho \phi_\mu^{A_t} \phi_\nu
+ 2 f_\gamma \rho_r \phi_{r \mu}^{A_t} \phi_{\nu}
+ 2 f_\gamma \rho_r \phi_{r \mu} \phi_{\nu}^{A_t}
\\ &\quad \mathrel+
\mathrm{interchange} (\mu, \nu)
%%
\\ &=
\frac{1}{2} f_{\rho \rho} \rho^{A_t} \cdot \phi_\mu \phi_\nu
+ f_{\rho \gamma} \rho_w \rho_w^{A_t} \phi_\mu \phi_\nu
+ 2 f_{\rho \gamma} \rho^{A_t} \rho_r \phi_{r \mu} \phi_{\nu}
+ 4 f_{\gamma \gamma} \rho_w \rho_w^{A_t} \rho_r \phi_{r \mu} \phi_{\nu}
+ 2 f_\gamma \rho_r^{A_t} \cdot \phi_{r \mu} \phi_{\nu}
%
\\ &\quad \mathrel-
f_\rho \phi_{t \mu_A} \phi_\nu
- 2 f_\gamma \rho_r \phi_{tr \mu_A} \phi_{\nu}
- 2 f_\gamma \rho_r \phi_{t \mu_A} \phi_{r \nu}
\\ &\quad \mathrel+
\mathrm{interchange} (\mu, \nu)
\end{align}

上面公式中，有一处看似错误的 $f_\gamma \rho_r \phi_{r \mu} \phi_{\nu}^{A_t}$ 到 $- f_\gamma \rho_r \phi_{t \mu_A} \phi_{r \nu}$ 的转换是有意为之，目的是为了程序书写的便利．这个转换的成立原因是利用 $\mathrm{interchange} (\mu, \nu)$．

现在我们考虑程序的编写．`Vxc_1_ao_contrib` 生成的是上式第二个等号的第二行，但去除了原子的限制条件

$$
- f_\rho \phi_{t \mu} \phi_\nu
- 2 f_\gamma \rho_r \phi_{tr \mu} \phi_{\nu}
- 2 f_\gamma \rho_r \phi_{t \mu} \phi_{r \nu}
$$

`Vxc_1_ao_` 一开始生成的是上式第二个等号的第一行；随后根据原子的限制条件，将 `Vxc_1_ao_contrib` 引入；最后与自己的转置进行加和，得到最终的交换相关势的一阶 Skeleton 导数．

In [13]:
Vxc_1_ao_contrib = (
    - np.einsum("g, tgu, gv -> tuv", kerh.fr, grdh.ao_1, grdh.ao_0)
    - 2 * np.einsum("g, rg, trgu, gv -> tuv", kerh.fg, grdh.rho_1, grdh.ao_2, grdh.ao_0)
    - 2 * np.einsum("g, rg, tgu, rgv -> tuv", kerh.fg, grdh.rho_1, grdh.ao_1, grdh.ao_1)
)

Vxc_1_ao_ = (
    + 0.5 * np.einsum("g, Atg, gu, gv -> Atuv", kerh.frr, grdh.A_rho_1, grdh.ao_0, grdh.ao_0)
    + np.einsum("g, wg, Atwg, gu, gv -> Atuv", kerh.frg, grdh.rho_1, grdh.A_rho_2, grdh.ao_0, grdh.ao_0)
    + 2 * np.einsum("g, Atg, rg, rgu, gv -> Atuv", kerh.frg, grdh.A_rho_1, grdh.rho_1, grdh.ao_1, grdh.ao_0)
    + 4 * np.einsum("g, wg, Atwg, rg, rgu, gv -> Atuv", kerh.fgg, grdh.rho_1, grdh.A_rho_2, grdh.rho_1, grdh.ao_1, grdh.ao_0)
    + 2 * np.einsum("g, Atrg, rgu, gv -> Atuv", kerh.fg, grdh.A_rho_2, grdh.ao_1, grdh.ao_0)
)

for A in range(natm):
    sA = mol_slice(A)
    Vxc_1_ao_[A, :, sA, :] += Vxc_1_ao_contrib[:, sA, :]
Vxc_1_ao_ += Vxc_1_ao_.swapaxes(-1, -2)

我们知道在 DFT 中，

$$
F_{\mu \nu} = h_{\mu \nu} + (\mu \nu | \kappa \lambda) D_{\kappa \lambda} - \frac{c_\mathrm{x}}{2} (\mu \kappa | \nu \lambda) D_{\kappa \lambda} + v_{\mu \nu}^{\mathrm{xc}} [\rho]
$$

因此非常简单地，

$$
F_{\mu \nu}^{A_t} = h_{\mu \nu}^{A_t} + (\mu \nu | \kappa \lambda)^{A_t} D_{\kappa \lambda} - \frac{c_\mathrm{x}}{2} (\mu \kappa | \nu \lambda)^{A_t} D_{\kappa \lambda} + v_{\mu \nu}^{\mathrm{xc}, A_t} [\rho]
$$

可以验证 GGAHelper 中 `F_1_ao` 变量即指代上述公式：

In [14]:
np.allclose(
    H_1_ao
    + np.einsum("Atuvkl, kl -> Atuv", eri1_ao, D)
    - 0.5 * cx * np.einsum("Atukvl, kl -> Atuv", eri1_ao, D)
    + Vxc_1_ao_,
    F_1_ao
)

True

在 PySCF 中，RHF 方法下同名的函数 `make_h1` 也能得到这个张量：

In [15]:
np.allclose(
    scf_hess.make_h1(C, mo_occ),
    F_1_ao
)

True

### $A_{pq, rs}$：计算

在给出 $A_{pq, rs}$ 的表达式之前，我们有必要简单回顾一下 CP-HF 方程是如何导出的．在 Yamaguchi (p129, 10.5) 中，我们知道了导出 CP-HF (类同于 CP-KS，因此这里不对两者的术语作区分) 方程的条件：

$$
(\partial_{A_t} + \partial_{A_t}^\mathrm{U}) F_{pq} = 0 \quad (p \neq q)
$$

从 CP-HF 方程 Yamaguchi (p437, X.1) 的衍生版本来看 (即将 $- \frac{1}{2} A_{pq, kl} S_{kl}^{A_t}$ 当作 $A_{pq, kl} U_{kl}^{A_t}$ 来处理，因为两者在对角标 $k, l$ 求和时等价)

$$
F_{pq}^{A_t} - S_{pq}^{A_t} \varepsilon_q + (\varepsilon_p - \varepsilon_q) U_{pq}^{A_t} + A_{pq, mk} U_{mk}^{A_t} = (\partial_{A_t} + \partial_{A_t}^\mathrm{U}) F_{pq}^{A_t} = 0
$$

从公式理解而非计算的角度来说，利用 $S_{pq}^{A_t} + U_{pq}^{A_t} + U_{qp}^{A_t} = 0$，我们还可以写为

$$
F_{pq}^{A_t} + U_{pq}^{A_t} \varepsilon_p + U_{qp}^{A_t} \varepsilon_q + A_{pq, mk} U_{mk}^{A_t} = 0
$$

我们对上式的 LHS 作下面的拆分：容易验证，

$$
\partial_{A_t} F_{pq} = F_{pq}^{A_t}
$$

以及 (注意到这里利用 Canonical HF 条件 $F_{pq} = \delta_{pq} \varepsilon_p$)

$$
F_{\mu \nu} \cdot \partial_{A_t}^\mathrm{U} (C_{\mu p} C_{\nu q}) = U_{mq}^{A_t} F_{pm} + U_{mp}^{A_t} F_{mq} = U_{pq}^{A_t} \varepsilon_p + U_{qp}^{A_t} \varepsilon_q
$$

因此，我们说，$A_{pq, mk} U_{mk}^{A_t}$ 的意义是

$$
C_{\mu p} C_{\nu q} \cdot \partial_{A_t}^\mathrm{U} F_{\mu \nu} = A_{pq, mk} U_{mk}^{A_t}
$$

可以看到，这个定义并不随着方法是 HF 或是 GGA 而变化．因此，我们一样能在 GGA 中定义 $A_{pq, mk} U_{mk}^{A_t}$，只是形式上与 HF 上会不同．

回顾到

$$
F_{\mu \nu} = h_{\mu \nu} + (\mu \nu | \kappa \lambda) D_{\kappa \lambda} - \frac{c_\mathrm{x}}{2} (\mu \kappa | \nu \lambda) D_{\kappa \lambda} + v_{\mu \nu}^\mathrm{xc} [\rho] 
$$

我们说，在 HF 中，

\begin{align}
\partial_{A_t}^\mathrm{U} F_{\mu \nu} &= (\mu \nu | \kappa \lambda) \cdot \partial_{A_t}^\mathrm{U} D_{\kappa \lambda} - \frac{1}{2} (\mu \kappa | \nu \lambda) \cdot \partial_{A_t}^\mathrm{U} D_{\kappa \lambda} \\
&= 2 (\mu \nu | \kappa \lambda) X_{\kappa \lambda} - (\mu \kappa | \nu \lambda) X_{\kappa \lambda}
\end{align}

上述的计算公式也是在 [RHF 实现](grad_rhf.ipynb#实现参考：四脚标-$A_{pq,-rs}$-函数) 中真正所使用的方法．那么对于 GGA，除了需要在交换积分上乘以系数 $c_\mathrm{x}$ 之外，还需要对 $v_{\mu \nu}^\mathrm{xc} [\rho]$ 作 U 偏导下的推演：

\begin{align}
\partial_{A_t}^\mathrm{U} v_{\mu \nu}^\mathrm{xc} [\rho] &= \frac{1}{2} \partial_{A_t}^\mathrm{U} f_\rho \cdot \phi_\mu \phi_\nu + 2 \partial_{A_t}^\mathrm{U} f_\gamma\cdot \rho_r \phi_{r \mu} \phi_{\nu} + 2 f_\gamma \cdot \partial_{A_t}^\mathrm{U} \rho_r \cdot \phi_{r \mu} \phi_{\nu} + \mathrm{interchange} (\mu, \nu) \\
&= X_{\kappa \lambda}^{A_t} (f_{\rho \rho} \phi_\kappa \phi_\lambda + 4 f_{\rho \gamma} \rho_w \phi_{w \kappa} \phi_\lambda) \phi_\mu \phi_\nu \\
&\quad \mathrel+ X_{\kappa \lambda}^{A_t} (4 f_{\rho \gamma} \phi_\kappa \phi_\lambda + 16 f_{\gamma \gamma} \rho_w \phi_{w \kappa} \phi_\lambda) \rho_r \phi_{r \mu} \phi_\nu \\
&\quad \mathrel+ X_{\kappa \lambda}^{A_t} (8 f_\gamma \phi_{r \kappa} \phi_\lambda) \phi_{r \mu} \phi_\nu \\
&\quad \mathrel+ \mathrm{interchange} (\mu, \nu)
\end{align}

在 PySCF 中，对于占据-非占轨道的 $A_{pi, qj} U_{qj}^{A_t}$，可以由 `hessian.rhf.gen_vind` 完成；尽管它是 RHF 下的函数，但具有计算 GGA 的能力．

In [16]:
hessian.rhf.gen_vind

<function pyscf.hessian.rhf.gen_vind(mf, mo_coeff, mo_occ)>

In [17]:
X = np.random.random((nmo, nocc))

dmX = C @ X @ Co.T
dmX += dmX.T
ax_ao = (
    + 1 * np.einsum("uvkl, kl -> uv", eri0_ao, dmX)
    - 0.5 * cx * np.einsum("ukvl, kl -> uv", eri0_ao, dmX)
    + np.einsum("kl, g, gk, gl, gu, gv -> uv", dmX, kerh.frr, grdh.ao_0, grdh.ao_0, grdh.ao_0, grdh.ao_0)
    + 4 * np.einsum("kl, g, wg, wgk, gl, gu, gv -> uv", dmX, kerh.frg, grdh.rho_1, grdh.ao_1, grdh.ao_0, grdh.ao_0, grdh.ao_0)
    + 4 * np.einsum("kl, g, gk, gl, rg, rgu, gv -> uv", dmX, kerh.frg, grdh.ao_0, grdh.ao_0, grdh.rho_1, grdh.ao_1, grdh.ao_0)
    + 16 * np.einsum("kl, g, wg, wgk, gl, rg, rgu, gv -> uv", dmX, kerh.fgg, grdh.rho_1, grdh.ao_1, grdh.ao_0, grdh.rho_1, grdh.ao_1, grdh.ao_0)
    + 8 * np.einsum("kl, g, rgk, gl, rgu, gv -> uv", dmX, kerh.fg, grdh.ao_1, grdh.ao_0, grdh.ao_1, grdh.ao_0)
)
ax_ao += ax_ao.T
np.allclose(C.T @ ax_ao @ Co, hessian.rhf.gen_vind(scf_eng, C, mo_occ)(X))

True

### $A_{pq, rs}$：性能优化

若在实际计算 $A_{pq, rs} U_{rs}^{A_t}$ 的过程直接代入 CP-HF 过程，计算耗时会比较多．因此，我们会打算对上述计算过程作一定程度的简化与优化．这部分的内容是参照了 PySCF 内部的运作流程而写的．

在对计算复杂度作评价时，尽管电子坐标维度 $r, w$ 在 `np.einsum_path` 中也属于是有效维度之一，但它不随着体系增大而增大，需要留意尽量不要让这个维度影响计算量的判断．当然，`np.einsum_path` 本身并不以维度数来判断那种张量缩并方案更好，而是通过当前张量的大小来判断．尽管不同大小的分子会有不同大小的张量，但一般来说，最优张量缩并的方案应当是相似的．

我们尽管会把原子核坐标分量 $A_t$ 记号写出，但它不参与计算量的判断中，因为我们不可能对 $A_t$ 进行计算上的缩并．因此，这一段中，$A_t$ 的数量当作 1．

首先，我们指出生成 $X_{\kappa \lambda}^{A_t} = U_{pi}^{A_t} (C_{\mu p} C_{\nu i} + C_{\mu i} C_{\nu p})$ 过程的计算复杂度是 $O(N^3)$，因为这是矩阵的连续相乘．其计算复杂度显然比 $(\mu \nu | \kappa \lambda) X_{\kappa \lambda}^{A_t}$ 等 ERI 缩并的复杂度小．

In [18]:
print(np.einsum_path("uvkl, kl -> uv", eri0_ao, dmX)[1])

  Complete contraction:  uvkl,kl->uv
         Naive scaling:  4
     Optimized scaling:  4
      Naive FLOP count:  4.685e+05
  Optimized FLOP count:  4.685e+05
   Theoretical speedup:  1.000
  Largest intermediate:  4.840e+02 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
   4                 kl,uvkl->uv                                   uv->uv


我们在这里指出，所有 GGA 格点积分的最优计算复杂度都要比 ERI 张量缩并小，且均为 $O(N^3)$，但这并不意味着计算量更小．拿格点积分中最简单的 $X_{\kappa \lambda}^{A_t} f_{\rho \rho} \phi_\kappa \phi_\lambda \phi_\mu \phi_\nu$ 来说，

In [19]:
print(np.einsum_path("kl, g, gk, gl, gu, gv -> uv", dmX, kerh.frr, grdh.ao_0, grdh.ao_0, grdh.ao_0, grdh.ao_0)[1])

  Complete contraction:  kl,g,gk,gl,gu,gv->uv
         Naive scaling:  5
     Optimized scaling:  3
      Naive FLOP count:  1.273e+11
  Optimized FLOP count:  1.834e+08
   Theoretical speedup:  694.435
  Largest intermediate:  1.993e+06 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
   2                    gk,g->kg                       kl,gl,gu,gv,kg->uv
   3                   gl,kl->kg                          gu,gv,kg,kg->uv
   2                    kg,kg->g                              gu,gv,g->uv
   2                    g,gu->ug                                gv,ug->uv
   3                   ug,gv->uv                                   uv->uv


其计算复杂度始终没有超过 $O(N^3)$；但其计算量远超过 ERI 的张量缩并；这是因为格点数量尽管随着原子数量的增长而线性地增长 (也近似于基组数量的增长趋势)；但格点数的增长比例非常高．我们可以粗略判断，在目前的 (75, 302) 格点下，当基组数超过下述值时，格点积分才体现出它的优势：

In [20]:
np.sqrt(grdh.ngrid / mol.natm)

150.4991694329241

我们目前的体系基组数是 22，显然远远不到上述的输出结果．

对于更多的计算复杂度的说明，可以交由读者自行查看．

我们应当留意到，由 `np.einsum_path` 给出的路径基本上都遵循先缩并 $\kappa, \lambda$ 的项，随后与格点以及 $\mu, \nu$ 的项进行缩并；但对每项分别进行独自的张量缩并会造成一些重复计算．因此，我们可以遵循以下两个原则，修改代码：

* 计算复杂度不超过 $O(N^3)$，并且该复杂度下的计算数次尽可能少；

* 计算复杂度为 $O(N^2)$ 的计算数次也应当尽可能少，但这可以宽松一些；

* 将所有可能重复使用的部分另存在内存中，但储存大小不应太大，否则内存 IO 时间会较多．

首先，我们生成广义密度矩阵 `rho_X_0` $\varrho = X_{\kappa \lambda}^{A_t} \phi_\kappa \phi_\lambda$ 与 `rho_X_1` $\varrho_r = 2 X_{\kappa \lambda}^{A_t} \phi_{r \kappa} \phi_\lambda$；同时出于简化目的，我们生成临时的 `gamma_XD` $\varrho_r \rho_r$，以及 `tmp_K` $K_\kappa = X_{\kappa \lambda}^{A_t} \phi_\lambda$．

使用 $K_\kappa$ 转存的目的确实是提高计算效率，因为在生成 $\varrho$ 与 $\varrho_r$ 时需要两次使用到 $K_\kappa$．

这一步的时间复杂度是 $O(N^2)$，为基组与格点的乘积．这一步的耗时在小体系下不是很小．

In [21]:
tmp_K = np.einsum("kl, gl -> gk", dmX, grdh.ao_0)
rho_X_0 = np.einsum("gk, gk -> g", grdh.ao_0, tmp_K)
rho_X_1 = 2 * np.einsum("rgk, gk -> rg", grdh.ao_1, tmp_K)
gamma_XD = np.einsum("rg, rg -> g", rho_X_1, grdh.rho_1)

随后，我们将与 $\mu, \nu$ 无关的信息全部计算完毕．我们期望生成一个张量 `tmp_M` $M$，使得 $v_{\mu \nu}^{A_t} = M \phi_{\nu} \phi_{\nu} + M_r \phi_{r \nu} \phi_{\nu}$．因此，

`tmp_M[0]`

$$
M = \varrho f_{\rho \rho} + 2 (\varrho_w \varrho_w) f_{\rho \gamma}
$$

`tmp_M[1:4]`

$$
M_r = 4 \varrho f_{\rho \gamma} \rho_r + 8 (\varrho_w \rho_w) f_{\gamma \gamma} \rho_r + 4 \varrho_r f_\gamma
$$

这一步的时间复杂度是 $O(N)$，为格点数．尽管这一步时间复杂度小，但由于通过归并同类项简化了后面的 O(N^3) 的计算量，因此是非常重要的一步．

In [22]:
tmp_M = np.empty((4, grdh.ngrid))
tmp_M[0] = (
    np.einsum("g, g -> g", rho_X_0, kerh.frr)
    + 2 * np.einsum("g, g -> g", gamma_XD, kerh.frg)
)
tmp_M[1:4] = (
    + 4 * np.einsum("g, g, rg -> rg", rho_X_0, kerh.frg, grdh.rho_1)
    + 8 * np.einsum("g, g, rg -> rg", gamma_XD, kerh.fgg, grdh.rho_1)
    + 4 * np.einsum("rg, g -> rg", rho_X_1, kerh.fg)
)

最后一步显然是 $v_{\mu \nu}^{A_t} = M \phi_{\nu} \phi_{\nu} + M_r \phi_{r \nu} \phi_{\nu}$．时间复杂度是 $O(N^3)$，为格点数与基组数平方的乘积．这一步只作一次，并且只在一行代码内实现：我们把 $M$ 与 $M_r$ 放在一起成为首维度为 4 的张量，并且注意到 $\phi_{\nu}$ 与 $\phi_{r \nu}$ 放在一起就是变量 `grdh.ao[0:4]`．

In [23]:
ax_ao = (
    + 1 * np.einsum("uvkl, kl -> uv", eri0_ao, dmX)
    - 0.5 * cx * np.einsum("ukvl, kl -> uv", eri0_ao, dmX)
    + np.einsum("rg, rgu, gv -> uv", tmp_M, grdh.ao[:4], grdh.ao_0)
)
ax_ao += ax_ao.T
np.allclose(C.T @ ax_ao @ Co, hessian.rhf.gen_vind(scf_eng, C, mo_occ)(X))

True

### $A_{pq, rs}$：效率评测

现在我们稍微小结一下刚才提到的一些程序方案，并对代码效率作简单的评测．

#### PySCF 做法

In [24]:
%%timeit -r 5 -n 30
hessian.rhf.gen_vind(scf_eng, C, mo_occ)(X)

300 ms ± 44.1 ms per loop (mean ± std. dev. of 5 runs, 30 loops each)


但这么评测是不太合适的，因为在通过 `hessian.rhf.gen_vind` 生成函数时也会产生一些计算；因此应当这么评测：

In [25]:
ax_pyscf = hessian.rhf.gen_vind(scf_eng, C, mo_occ)

In [26]:
%%timeit -r 5 -n 30
ax_pyscf(X)

118 ms ± 33 ms per loop (mean ± std. dev. of 5 runs, 30 loops each)


#### 原始做法

原始做法之所以耗时，应当是因为它进行了五次 $O(N^3)$ 的计算量：

In [27]:
%%timeit -r 5 -n 30
dmX = C @ X @ Co.T
dmX += dmX.T
ax_ao = (
    + 1 * np.einsum("uvkl, kl -> uv", eri0_ao, dmX)
    - 0.5 * cx * np.einsum("ukvl, kl -> uv", eri0_ao, dmX)
    + np.einsum("kl, g, gk, gl, gu, gv -> uv", dmX, kerh.frr, grdh.ao_0, grdh.ao_0, grdh.ao_0, grdh.ao_0)
    + 4 * np.einsum("kl, g, wg, wgk, gl, gu, gv -> uv", dmX, kerh.frg, grdh.rho_1, grdh.ao_1, grdh.ao_0, grdh.ao_0, grdh.ao_0)
    + 4 * np.einsum("kl, g, gk, gl, rg, rgu, gv -> uv", dmX, kerh.frg, grdh.ao_0, grdh.ao_0, grdh.rho_1, grdh.ao_1, grdh.ao_0)
    + 16 * np.einsum("kl, g, wg, wgk, gl, rg, rgu, gv -> uv", dmX, kerh.fgg, grdh.rho_1, grdh.ao_1, grdh.ao_0, grdh.rho_1, grdh.ao_1, grdh.ao_0)
    + 8 * np.einsum("kl, g, rgk, gl, rgu, gv -> uv", dmX, kerh.fg, grdh.ao_1, grdh.ao_0, grdh.ao_1, grdh.ao_0)
)
ax_ao += ax_ao.T
C.T @ ax_ao @ Co

275 ms ± 23.2 ms per loop (mean ± std. dev. of 5 runs, 30 loops each)


#### 改进做法

改进做法尽管耗时少了许多，但代码可读性也相对地变差了一些．

但代码可读性的变差是相对的．我们将来会推导 GGA 的 $A_{pq, rs}^{A_t} X_{rs}$ 的表达式，届时将会发现这种改进做法不仅在代码效率上有所提高，对公式推导也有帮助．

In [28]:
%%timeit -r 5 -n 30
dmX = C @ X @ Co.T
dmX += dmX.T
tmp_K = np.einsum("kl, gl -> gk", dmX, grdh.ao_0)
rho_X_0 = np.einsum("gk, gk -> g", grdh.ao_0, tmp_K)
rho_X_1 = 2 * np.einsum("rgk, gk -> rg", grdh.ao_1, tmp_K)
gamma_XD = np.einsum("rg, rg -> g", rho_X_1, grdh.rho_1)
tmp_M = np.empty((4, grdh.ngrid))
tmp_M[0] = (
    np.einsum("g, g -> g", rho_X_0, kerh.frr)
    + 2 * np.einsum("g, g -> g", gamma_XD, kerh.frg)
)
tmp_M[1:4] = (
    + 4 * np.einsum("g, g, rg -> rg", rho_X_0, kerh.frg, grdh.rho_1)
    + 8 * np.einsum("g, g, rg -> rg", gamma_XD, kerh.fgg, grdh.rho_1)
    + 4 * np.einsum("rg, g -> rg", rho_X_1, kerh.fg)
)
ax_ao = (
    + 1 * np.einsum("uvkl, kl -> uv", eri0_ao, dmX)
    - 0.5 * cx * np.einsum("ukvl, kl -> uv", eri0_ao, dmX)
    + np.einsum("rg, rgu, gv -> uv", tmp_M, grdh.ao[:4], grdh.ao_0)
)
ax_ao += ax_ao.T
C.T @ ax_ao @ Co

91.6 ms ± 34.8 ms per loop (mean ± std. dev. of 5 runs, 30 loops each)


## 实现参考：二阶导数性质

### GGA 二阶梯度 $\frac{\partial^2}{\partial A_t \partial B_s} E_\mathrm{elec}$

在继续梯度的代码实现之前，我们需要对梯度的推导作一个简短回顾．

我们知道，一阶梯度的计算包含以下两部分：

$$
\frac{\partial}{\partial A_t} E_\mathrm{elec} = \partial_{A_t} E_\mathrm{elec} + \partial_{A_t}^\mathrm{U} E_\mathrm{elec}
$$

$\partial_{A_t}^\mathrm{U}$ 偏导数显然会出现 U 矩阵；但之所以一阶梯度没有 U 矩阵的贡献，是因为所有 U 矩阵可以依靠求和时的对称性化为 $S_{pq}^{A_t}$．事实上，我们可以将上式写为 (对于非自洽亦是类似的；对于 $\partial_{A_t}^\mathrm{U} E_\mathrm{elec} = 4 U_{pi} F_{pi}^{A_t}$ 的合理性，请参考后续的 [非自洽一阶梯度笔记](grad_hf_ncgga.ipynb#U-矩阵直接求解))

$$
\frac{\partial}{\partial A_t} E_\mathrm{elec} = \partial_{A_t} E_\mathrm{elec} + 4 U_{pi} F_{pi}^{A_t}
$$

那么，二阶梯度可以写为

\begin{align}
\frac{\partial^2}{\partial A_t \partial B_s} E_\mathrm{elec}
&= \partial_{A_t} \partial_{B_s} E_\mathrm{elec} + \partial_{A_t}^\mathrm{U} \partial_{B_s} E_\mathrm{elec} + \partial_{A_t} \partial_{B_s}^\mathrm{U} E_\mathrm{elec} + \partial_{A_t}^\mathrm{U} \partial_{B_s}^\mathrm{U} E_\mathrm{elec} \\
&= \partial_{A_t} \partial_{B_s} E_\mathrm{elec} + 4 U_{pi}^{A_t} F_{pi}^{B_s} + 4 U_{pi}^{B_s} F_{pi}^{A_t} + 4 \partial_{B_s}^\mathrm{U} (U_{pi}^{A_t} F_{pi}) \\
&= \partial_{A_t} \partial_{B_s} E_\mathrm{elec} + 4 U_{pi}^{A_t} F_{pi}^{B_s} + 4 U_{pi}^{B_s} F_{pi}^{A_t} + 4 U_{pi}^{A_t} U_{qp}^{B_s} F_{qi} + 4 U_{pi}^{A_t} U_{qi}^{B_s} F_{pq} + 4 U_{pi}^{A_t B_s} F_{pi} - 4 U_{pq}^{B_s} U_{qi}^{A_t} F_{pi} + 4 U_{pi}^{A_t} A_{pi, qj} U_{qj}^{B_s} \\
&= \partial_{A_t} \partial_{B_s} E_\mathrm{elec} + 4 U_{pi}^{A_t} F_{pi}^{B_s} + 4 U_{pi}^{B_s} F_{pi}^{A_t} + 4 U_{pi}^{A_t} U_{qi}^{B_s} F_{pq} + 4 U_{pi}^{A_t B_s} F_{pi} + 4 U_{pi}^{A_t} A_{pi, qj} U_{qj}^{B_s}
\end{align}

<div class="alert alert-info">

**任务**

1. 请不要假设 $F_{pq} = \delta_{pq} \varepsilon_p$，而将 $F_{pq}$ 暂时看成一个稠密矩阵，验证上述等式．这部分的推导事实上与非自洽泛函的推导是相同的．

</div>

而我们从 Yamaguchi (p428, V.2) 中，知道

\begin{align}
\frac{\partial^2}{\partial A_t \partial B_s} E_\mathrm{elec}
&= \partial_{A_t} \partial_{B_s} E_\mathrm{elec} - 2 \xi_{ii}^{A_t B_s} \varepsilon_i \\
&\quad + 4 U_{pi}^{B_s} F_{pi}^{A_t} + 4 U_{pi}^{A_t} F_{pi}^{B_s} + 4 U_{pi}^{A_t} U_{pi}^{B_s} \varepsilon_p + 4 U_{pi}^{A_t} A_{pi, qj} U_{qj}^{B_s}
\end{align}

如果引入 $F_{pq} = \delta_{pq} \varepsilon_p$，我们就会知道

* $4 U_{pi}^{A_t B_s} F_{pi} = - 2 \xi_{ii}^{A_t B_s} \varepsilon_i$

* $4 U_{pi}^{A_t} U_{qi}^{B_s} F_{pq} = 4 U_{pi}^{A_t} U_{pi}^{B_s} \varepsilon_p$

因此，上述的推导与 Yamaguchi 的结论是可以进行比较的．

而既然我们已经定义好了 GGA 的 Fock 矩阵，因此 HF 公式中凡是涉及到 Fock 矩阵部分的，我们都可以用 GGA 的 Fock 矩阵替代；这对于 $A_{pi, qj}$ 张量与由此而来的 $U_{pi}^{A_t}$ 亦如此．故而，二阶梯度公式中，HF 与 GGA 唯有在 $\partial_{A_t} \partial_{B_s} E_\mathrm{elec}$，公式形式有所差别．

在 GGA 中，

$$
\partial_{A_t} \partial_{B_s} E_\mathrm{elec} = h_{\mu \nu}^{A_t B_s} D_{\mu \nu} + \frac{1}{2} (\mu \nu | \kappa \lambda)^{A_t B_s} D_{\mu \nu} D_{\kappa \lambda} - \frac{c_\mathrm{x}}{4} (\mu \kappa | \nu \lambda)^{A_t B_s} D_{\mu \nu} D_{\kappa \lambda} + \partial_{A_t} \partial_{B_s} (f \rho)
$$

其中的交换相关能贡献是

\begin{align}
\partial_{A_t} \partial_{B_s} (f \rho) &= \partial_{A_t} (f_\rho \rho^{B_s} + 2 f_\gamma \rho_r \rho_r^{B_s}) \\
&= f_{\rho \rho} \rho^{A_t} \rho^{B_s} + 2 f_{\rho \gamma} \rho_w \rho_w^{A_t} \rho^{B_s} + f_\rho \rho^{A_t B_s} \\
&\quad + 2 f_{\rho \gamma} \rho^{A_t} \rho_r \rho_r^{B_s} + 4 f_{\gamma \gamma} \rho_w \rho_w^{A_t} \rho_r \rho_r^{B_s} + 2 f_\gamma \rho_r^{A_t} \rho_r^{B_s} + 2 f_\gamma \rho_r \rho_r^{A_t B_s}
\end{align}

对于上式的计算，我们指出，其中不包含 $\rho^{A_t B_s}$ 的项可以很容易地生成．我们已经在 `grdh.A_rho_1` 与 `grdh.A_rho_2` 中给出了 $\rho^{A_t}$ 与 $\rho_r^{A_t}$，但我们不打算再储存 $\rho^{A_t B_s}$ 或 $\rho_r^{A_t B_s}$；这主要是因为储存 $\rho^{A_t B_s}$ 随着体系增大而呈三次方增长；我们不希望储存这么庞大的矩阵 (事实上 ERI 张量是四次方的，但这里为了小体系的计算效率与使用上的便利我们先存下来)．

我们简单地导出 $\rho^{A_t B_s}$ 与 $\rho_r^{A_t B_s}$．下面会引入记号 $T = ts$，它同时代表 $t, s$ 维度．因为我们生成出来的原子轨道格点张量具有对称性 $\phi_{ts \mu} = \phi_{st \mu}$，因此原来 9 维大小的 $t, s$ 实际上只有 $3 \times (3 + 1) / 2 = 6$ 维的信息量；并且在一部分张量缩并时，$t, s$ 维度或者同时保留或者同时消去，因此我们可以将 9 维的 $t, s$ 使用六维的 $T$ 替代．

\begin{align}
\rho^{A_t B_s} &= 2 D_{\mu \nu} \partial_{B_s} (\phi_\mu^{A_t} \phi_\nu) = 2 D_{\mu \nu} (\phi_\mu^{A_t B_s} \phi_\nu + \phi_\mu^{A_t} \phi_\nu^{B_s}) \\
&= 2 D_{\mu_{AB} \nu} \phi_{T \mu} \phi_\nu + 2 D_{\mu_A \nu_B} \phi_{t \mu} \phi_{s \nu}
\end{align}

\begin{align}
\rho_r^{A_t B_s} &= 2 D_{\mu \nu} \partial_{B_s} (\phi_{r \mu}^{A_t} \phi_\nu + \phi_\mu^{A_t} \phi_{r \nu}) = 2 D_{\mu \nu} (\phi_{r \mu}^{A_t B_s} \phi_\nu + \phi_\mu^{A_t B_s} \phi_{r \nu} + \phi_{r \mu}^{A_t} \phi_\nu^{B_s} + \phi_\mu^{A_t} \phi_{r \nu}^{B_s}) \\
&= 2 D_{\mu_{AB} \nu} (\phi_{Tr \mu} \phi_\nu + \phi_{T \mu} \phi_{r \nu}) + 2 D_{\mu_A \nu_B} (\phi_{tr \mu} \phi_{s \nu} + \phi_{t \mu} \phi_{sr \nu})
\end{align}

我们将 $\rho^{A_t B_s}$ 与 $f_\rho$ 相乘，将 $\rho_r^{A_t B_s}$ 与 $2 f_\gamma \rho_r$ 相乘，就得到

\begin{align}
f_\rho \rho^{A_t B_s} + 2 f_\gamma \rho_r \rho_r^{A_t B_s}
&= D_{\mu_{AB} \nu} (2 f_\rho \phi_{T \mu} \phi_\nu + 4 f_\gamma \rho_r \phi_{Tr \mu} \phi_\nu + 4 f_\gamma \rho_r \phi_{T \mu} \phi_{r \nu}) \\
&\quad \mathrel+ D_{\mu_A \nu_B} (2 f_\rho \phi_{t \mu} \phi_{s \nu} + 4 f_\gamma \rho_r \phi_{tr \mu} \phi_{s \nu} + 4 f_\gamma \rho_r \phi_{t \mu} \phi_{sr \nu})
\end{align}

下面我们来着手实现 GGA 的二阶梯度中，$\partial_{A_t} \partial_{B_s} E_\mathrm{elec}$ 的结果的重复．

下个代码块解决 $\partial_{A_t} \partial_{B_s} (f \rho)$ 中，关于 $D_{\mu_{AB} \nu}$ 的部分．

* `tmp_tensor_1` $2 f_\rho \phi_{T \mu} \phi_\nu + 4 f_\gamma \rho_r \phi_{Tr \mu} \phi_\nu + 4 f_\gamma \rho_r \phi_{T \mu} \phi_{r \nu}$

* `E_SS_GGA_contrib1` $D_{\mu_{AB} \nu} (2 f_\rho \phi_{T \mu} \phi_\nu + 4 f_\gamma \rho_r \phi_{Tr \mu} \phi_\nu + 4 f_\gamma \rho_r \phi_{T \mu} \phi_{r \nu})$

这其中用到了 $T$ 与 $ts$ 的关系．

In [29]:
tmp_tensor_1 = (
    + 2 * np.einsum("g, Tgu, gv -> Tuv", kerh.fr, grdh.ao_2T, grdh.ao_0)
    + 4 * np.einsum("g, rg, rTgu, gv -> Tuv", kerh.fg, grdh.rho_1, grdh.ao_3T, grdh.ao_0)
    + 4 * np.einsum("g, rg, Tgu, rgv -> Tuv", kerh.fg, grdh.rho_1, grdh.ao_2T, grdh.ao_1)
)
XX, XY, XZ, YY, YZ, ZZ = range(6)
E_SS_GGA_contrib1 = np.zeros((natm, natm, 3, 3))
for A in range(natm):
    sA = mol_slice(A)
    E_SS_GGA_contrib1[A, A] = np.einsum("Tuv, uv -> T", tmp_tensor_1[:, sA], D[sA])[[XX, XY, XZ, XY, YY, YZ, XZ, YZ, ZZ]].reshape(3, 3)

下个代码块解决 $\partial_{A_t} \partial_{B_s} (f \rho)$ 中，关于 $D_{\mu_A \nu_B}$ 的部分．

* `tmp_tensor_2` $2 f_\rho \phi_{t \mu} \phi_{s \nu} + 4 f_\gamma \rho_r \phi_{tr \mu} \phi_{s \nu} + 4 f_\gamma \rho_r \phi_{t \mu} \phi_{sr \nu}$

* `E_SS_GGA_contrib2` $D_{\mu_A \nu_B} (2 f_\rho \phi_{t \mu} \phi_{s \nu} + 4 f_\gamma \rho_r \phi_{tr \mu} \phi_{s \nu} + 4 f_\gamma \rho_r \phi_{t \mu} \phi_{sr \nu})$

其中利用到 $f_\gamma \rho_r \phi_{tr \mu} \phi_{s \nu}$ 与 $f_\gamma \rho_r \phi_{t \mu} \phi_{sr \nu}$ 之间恰好是 $t \mu$ 与 $s \nu$ 的转换．事实上，生成 $f_\gamma \rho_r \phi_{tr \mu} \phi_{s \nu}$ 将是这其中计算量相对较大的一步，因此能省这一步就省出这一步．当然，也许我们也会说这没有影响到计算量级的变化，并且生成梯度的过程中，这步计算只需要执行一次，因此也不是非常打紧．

In [30]:
tmp_tensor_2 = 4 * np.einsum("g, rg, trgu, sgv -> tsuv", kerh.fg, grdh.rho_1, grdh.ao_2, grdh.ao_1)
tmp_tensor_2 += tmp_tensor_2.transpose(1, 0, 3, 2)
tmp_tensor_2 += 2 * np.einsum("g, tgu, sgv -> tsuv", kerh.fr, grdh.ao_1, grdh.ao_1)
E_SS_GGA_contrib2 = np.empty((natm, natm, 3, 3))
for A in range(natm):
    sA = mol_slice(A)
    for B in range(A + 1):
        sB = mol_slice(B)
        E_SS_GGA_contrib2[A, B] = np.einsum("tsuv, uv -> ts", tmp_tensor_2[:, :, sA, sB], D[sA, sB])
        if A != B:
            E_SS_GGA_contrib2[B, A] = E_SS_GGA_contrib2[A, B].T

下个代码块解决 $\partial_{A_t} \partial_{B_s} (f \rho)$ 中，可以直接生成的部分．由于这里不牵涉到基组，因此其计算量相对较小．

* `E_SS_GGA_contrib3` $f_{\rho \rho} \rho^{A_t} \rho^{B_s} + 2 f_{\rho \gamma} \rho_w \rho_w^{A_t} \rho^{B_s} + 2 f_{\rho \gamma} \rho^{A_t} \rho_r \rho_r^{B_s} + 4 f_{\gamma \gamma} \rho_w \rho_w^{A_t} \rho_r \rho_r^{B_s} + 2 f_\gamma \rho_r^{A_t} \rho_r^{B_s}$

In [31]:
E_SS_GGA_contrib3 = (
    + np.einsum("g, Atg, Bsg -> ABts", kerh.frr, grdh.A_rho_1, grdh.A_rho_1)
    + 2 * np.einsum("g, wg, Atwg, Bsg -> ABts", kerh.frg, grdh.rho_1, grdh.A_rho_2, grdh.A_rho_1)
    + 2 * np.einsum("g, Atg, rg, Bsrg -> ABts", kerh.frg, grdh.A_rho_1, grdh.rho_1, grdh.A_rho_2)
    + 4 * np.einsum("g, wg, Atwg, rg, Bsrg -> ABts", kerh.fgg, grdh.rho_1, grdh.A_rho_2, grdh.rho_1, grdh.A_rho_2)
    + 2 * np.einsum("g, Atrg, Bsrg -> ABts", kerh.fg, grdh.A_rho_2, grdh.A_rho_2)
)

至此，GGA 部分的计算就已经结束了．你也许可以看看每一个 GGA 贡献分项的大小，会发现这些贡献分项经常会非常大．

下个代码块解决 HF 部分．

* `E_SS_HF_contrib` $h_{\mu \nu}^{A_t B_s} D_{\mu \nu} + \frac{1}{2} (\mu \nu | \kappa \lambda)^{A_t B_s} D_{\mu \nu} D_{\kappa \lambda} - \frac{c_\mathrm{x}}{4} (\mu \kappa | \nu \lambda)^{A_t B_s} D_{\mu \nu} D_{\kappa \lambda}$

In [32]:
E_SS_HF_contrib = (
    + np.einsum("ABtsuv, uv -> ABts", H_2_ao, D)
    + 0.5 * np.einsum("ABtsuvkl, uv, kl -> ABts", eri2_ao, D, D)
    - 0.25 * cx * np.einsum("ABtsukvl, uv, kl -> ABts", eri2_ao, D, D)
)

在 PySCF 中，所有与 U 矩阵无关的项会通过 `hessian.rks.partial_hess_elec` 函数来完成；但该函数还包括了 $-2 S_{ii}^{A_t} \varepsilon_i$ 的贡献；它实际上是从二阶 U 矩阵衍生而来．我们可以核准其正确性．

In [33]:
scf_hess.partial_hess_elec.__func__

<function pyscf.hessian.rks.partial_hess_elec(hessobj, mo_energy=None, mo_coeff=None, mo_occ=None, atmlst=None, max_memory=4000, verbose=None)>

In [34]:
np.allclose(
    E_SS_GGA_contrib1 + E_SS_GGA_contrib2 + E_SS_GGA_contrib3 + E_SS_HF_contrib
    - 2 * np.einsum("ABtsuv, ui, vi, i -> ABts", S_2_ao, Co, Co, eo),
    scf_hess.partial_hess_elec()
)

True

<div class="alert alert-info">

**任务**

1. (可选) 事实上，我们的代码实现效率在当前体系下比 PySCF (v1.6) 的快不少．请 Hack PySCF 的代码，并指出原因．

   * 你应当能发现生成 `E_SS_GGA_contrib2` 结果的效率远超过 PySCF．请找到 PySCF 中生成这部分、以及 `E_SS_GGA_contrib1` 贡献之和的 PySCF 函数，并作运行时间测评．

</div>

有了上面的结论后，我们就可以很容易地生成二阶梯度了．

\begin{align}
\frac{\partial^2}{\partial A_t \partial B_s} E_\mathrm{elec}
&= \partial_{A_t} \partial_{B_s} E_\mathrm{elec} - 2 \xi_{ii}^{A_t B_s} \varepsilon_i \\
&\quad + 4 U_{pi}^{B_s} F_{pi}^{A_t} + 4 U_{pi}^{A_t} F_{pi}^{B_s} + 4 U_{pi}^{A_t} U_{pi}^{B_s} \varepsilon_p + 4 U_{pi}^{A_t} A_{pi, qj} U_{qj}^{B_s}
\end{align}

In [35]:
np.allclose(
    E_SS_GGA_contrib1 + E_SS_GGA_contrib2 + E_SS_GGA_contrib3 + E_SS_HF_contrib
    - 2 * np.einsum("ABtsi, i -> ABts", Xi_2.diagonal(0, 4, 5)[:, :, :, :, so], eo)
    + 4 * np.einsum("Bspi, Atpi -> ABts", U_1[:, :, :, so], F_1_mo[:, :, :, so])
    + 4 * np.einsum("Atpi, Bspi -> ABts", U_1[:, :, :, so], F_1_mo[:, :, :, so])
    + 4 * np.einsum("Atpi, Bspi, p -> ABts", U_1[:, :, :, so], U_1[:, :, :, so], e)
    + 4 * np.einsum("Atpi, Bspi -> ABts", U_1[:, :, :, so], Ax0_Core(sa, so, sa, so)(U_1[:, :, :, so]))
    + scf_hess.hess_nuc(),
    scf_hess.de
)

True

## 未完待续

我们至少还需要生成 GGA 的二阶 U 矩阵．